In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from z3 import Int, Solver, sat
from spytial import diagram
from spytial.annotations import orientation, attribute

# Define a simple Z3 problem
x = Int('x')
y = Int('y')
s = Solver()
s.add(x + y == 10, x > 0, y > 0)

if s.check() == sat:  # Compare with z3.sat
    z3_model = s.model()
    print("Z3 Model:")
    
    diagram(z3_model)
else:
    print(s.check())
    print("No solution found.")

Z3 Model:


# But is this useful? Not really

You can register a DataInstanceBuilder for Z3 that provides some directions on HOW things should be translated to boxes and arrows.



In [3]:
from spytial.provider_system import DataInstanceProvider, data_provider
from z3 import ModelRef

# Custom provider for Z3 models
@data_provider(priority=10)
class Z3ModelProvider(DataInstanceProvider):
    """Custom provider for Z3 models."""

    def can_handle(self, obj):
        return isinstance(obj, ModelRef)  # Check if the object is a Z3 model

    def provide_atoms_and_relations(self, obj, walker_func):
        atoms = []
        relations = []

        for decl in obj.decls():
            var_name = decl.name()
            var_value = obj.eval(decl, model_completion=True)  # Ensure proper evaluation
            atom_id = walker_func._get_id(var_name)
            atom = {
                "id": atom_id,
                "type": "Z3Variable",
                "label": f"{var_name} = {var_value}"
            }
            atoms.append(atom)

            # Example: Add a dummy relation for demonstration (can be extended)
            if isinstance(var_value, int):
                relations.append(("depends_on", atom_id, walker_func._get_id("constant")))

        # Add a dummy constant atom for demonstration
        constant_atom = {
            "id": walker_func._get_id("constant"),
            "type": "Constant",
            "label": "Constant Value"
        }
        atoms.append(constant_atom)

        return atoms, relations


# Register the custom provider
provider = Z3ModelProvider()


# Now visualize again
diagram(z3_model)

Error during data instance building: b'ast is not an expression'
